In [1]:
!pip install transformers
!pip install gradio
!pip install timm
!pip install inflect
!pip install phonemizer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 32.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 19.7 MB/s eta 0:00:00 0:00:01
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
  Attempting uninstall: aiofiles
    Found existing installation: aiofiles 24.1.0
    Uninstalling aiofiles-24.1.0:
      Successfully uninstalled aiofiles-24.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unstructured-client 0.28.1 requires aiofiles>=24.1.0, but you have aiofiles 23.2.1 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 31.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━

In [4]:
!sudo apt-get update
!sudo apt-get install espeak-ng
!pip install py-espeak-ng

Get:1 file:/var/cuda-repo-ubuntu2204-12-6-local  InRelease [1572 B]
Get:1 file:/var/cuda-repo-ubuntu2204-12-6-local  InRelease [1572 B]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease                   
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease         
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  espeak-ng-data libasyncns0 libespeak-ng1 libflac8 libogg0 libopus0
  libpcaudio0 libpulse0 libsndfile1 libsonic0 libvorbis0a libvorbisenc2
Suggested packages:
  opus-tools pulseaudio
The following NEW packages will be installed:
  espeak-ng espeak-ng-data libasyncns0 libespeak-ng1 libflac8 libogg0 libopus0
  libpcaudio0 libpulse0 libsndfile1 libsonic0 libvorbis0a libvorbisenc2
0 upgraded, 13 newly insta

In [5]:
# 불필요한 Warning 제거
from transformers.utils import logging
logging.set_verbosity_error()

- conditionalGeneration : warm 상태의 모델 불러올 때 이용

In [6]:
from transformers import MusicgenForConditionalGeneration

model = MusicgenForConditionalGeneration.from_pretrained("facebook/musicgen-small")

2024-12-07 00:17:25.330419: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733498245.469456    3091 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733498245.511055    3091 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-07 00:17:25.838590: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


config.json:   0%|          | 0.00/7.87k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.36G [00:00<?, ?B/s]

/home/tommy/anaconda3/envs/transformer/lib/python3.11/site-packages/transformers/models/encodec/modeling_encodec.py:124: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("padding_total", torch.tensor(kernel_size - stride, dtype=torch.int64), persistent=False)


generation_config.json:   0%|          | 0.00/224 [00:00<?, ?B/s]

In [7]:
import torch

print("model == ", model)
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model.to(device);

model ==  MusicgenForConditionalGeneration(
  (text_encoder): T5EncoderModel(
    (shared): Embedding(32128, 768)
    (encoder): T5Stack(
      (embed_tokens): Embedding(32128, 768)
      (block): ModuleList(
        (0): T5Block(
          (layer): ModuleList(
            (0): T5LayerSelfAttention(
              (SelfAttention): T5Attention(
                (q): Linear(in_features=768, out_features=768, bias=False)
                (k): Linear(in_features=768, out_features=768, bias=False)
                (v): Linear(in_features=768, out_features=768, bias=False)
                (o): Linear(in_features=768, out_features=768, bias=False)
                (relative_attention_bias): Embedding(32, 12)
              )
              (layer_norm): T5LayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (1): T5LayerFF(
              (DenseReluDense): T5DenseActDense(
                (wi): Linear(in_features=768, out_features=3072, bias=False)
             

In [18]:
# Conditioned Generation을 위한 입력 생성

from transformers import AutoProcessor

processor = AutoProcessor.from_pretrained("facebook/musicgen-small")

inputs = processor(
    # text=["80s pop track with bassy drums and synth", "90s rock song with loud guitars and heavy drums"],
    text=["i want you to make modal piano bibab jazz"],
    padding=True,
    return_tensors="pt",
)

In [19]:
# 음악 생성
generated_music = model.generate(**inputs.to(device), do_sample=True, guidance_scale=3, max_new_tokens=1024)

In [20]:
### 생성된 음악 저장
import scipy

sampling_rate = model.config.audio_encoder.sampling_rate
audio_data=generated_music[0, 0].cpu().numpy()

f_audio = "musicgen_out.wav"
scipy.io.wavfile.write(f_audio, rate=sampling_rate, data = audio_data )

In [21]:
# 생성된 음악 재생

from IPython.display import display, Audio


audio = Audio(audio_data, rate=sampling_rate)
display(audio)

In [22]:
# !pip install librosa

In [23]:
import librosa
import numpy as np


audio_file = f_audio
audio_data, sampling_rate = librosa.load(audio_file)

# Normalize the audio data to the range [-1, 1]
audio_data = audio_data / np.max(np.abs(audio_data))

audio = Audio(audio_data, rate=sampling_rate)
display(audio)